#RNN

##import/load

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print('Length of text: {} characters'.format(len(text)))

1122304/1115394 [==============================] - 0s 0us/step
Length of text: 1115394 characters


In [3]:
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



##vectorize

In [4]:
# The unique characters
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

65 unique characters


In [5]:
# map unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [6]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [7]:
# Show how the first 13 characters from the text are mapped to integers
print('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [8]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [9]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [12]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


##base model

In [13]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

In [14]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [15]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [16]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [17]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [19]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [20]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 ' which was to be seen,\ncannot be spoken of. There might you have beheld one\njoy crown another, so an'

Next Char Predictions: 
 "\nxP;'PLqkdVhRvA\n;sXpAaORt?WgHeDD;gXMW:pbGnJbtFURcKn'MBawfCdWw?O;rOl?TOYA,BLpTLty$LkgOj;KHV$j-WqfuX\nx"


In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.1746807


In [22]:
model.compile(optimizer='adam', loss=loss)

## epoch

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [24]:
EPOCHS = 10

In [25]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 12s 58ms/step - loss: 3.2720
Epoch 2/10
172/172 [==============================] - 11s 58ms/step - loss: 2.0511
Epoch 3/10
172/172 [==============================] - 11s 59ms/step - loss: 1.7427
Epoch 4/10
172/172 [==============================] - 11s 59ms/step - loss: 1.5733
Epoch 5/10
172/172 [==============================] - 11s 59ms/step - loss: 1.4704
Epoch 6/10
172/172 [==============================] - 11s 58ms/step - loss: 1.3988
Epoch 7/10
172/172 [==============================] - 11s 58ms/step - loss: 1.3525
Epoch 8/10
172/172 [==============================] - 11s 58ms/step - loss: 1.3138
Epoch 9/10
172/172 [==============================] - 11s 58ms/step - loss: 1.2746
Epoch 10/10
172/172 [==============================] - 11s 58ms/step - loss: 1.2393


In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_10'

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


## generate

In [29]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [30]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: he is like a saith,
And le in that doth she's a little spirit with shroud
Weeping wind Romeo; you are so, bring his charms.

GLOUCESTER:
Why, closely-tongues. No more of my tongue?
Dost look your whose soveries in the trumpets born,
To find some finning and sovereigns, who should reliss our petnicon
Romeo all hours in good honourable from a famous!
By not a seit doth same heir
For henter this mouther there's than direction of it:
Whither I should not have some windows in the crown,
Whilvis disgail you with Norfolk: the grey comnock, he madestake
A hat, alone;
We should not be better suit,
Envive enforningment a view; for if he chinsimes
Listor to the sea, and utter--
Shall I from he violence intends and Henry Mine.

MARIANA:
Now the god of Mursuas, if thou liest? I had as
unle assurenced their fair swift and on the sky, of him that
A little office, to the kingdom,--
We have we two of you born--not under fellow, my joots,
And making down of such a child,
To suck then well; thou n

In [31]:
print(generate_text(model, start_string=u"Whence"))

Whence shall estail unhaven by his cale.
Well, no more striam be joughter heme, what but the mothing is you didst,
For not to ask thy infucinian false froands;
How all the issuind our hands of sixteen Henry will corners obplain on it; and a patience
If I abrude of thy nobals?

WARWICK:
BALTHIMANIE:
Pompey! the great will I lead do say.

ROMEO:
On past out my good lord, thou on my word; I would not
There is an unneated world with merry, and thou wilt.

LADY ANNE:
If for a seizar shall say.

DUCHESS OF YORK:
Shall be think'd in thine eye betwixt me but
dewine and dreadf. Yet send
And sperbleing wakes within thy high adventable bock;
And therefore preshall slaid my father with her sight!
I'll deling you to let him to the sting.
He cried; it tooking on their displexeth accessagear.

MISTRESS OVERDONE:
No.

ESCALUS:
Vecund, was this worth'd riches of chain with us,
About the lurefore deed words and your execess by not-inllowledge.

Second Husting arr bound with child
Much is our blood upon


In [32]:
print(generate_text(model, start_string=u"Love"))

Love County!' be gone; adwick up: he will bring me
To do some name for is end.
We the exacen east on too,
And in the virtues shut'stroord seas!

ISABELLA:
I may glad it eath perforch of motion to pity his master's infany. Here is with her?

POMPEY:
Art thou, hopy which now is thy rife:
She comest toucheth truly in fashion of not pois,
I hear them fell for what think you to be interringmen:
I dorr forcome to veam, of faces pray with parason prays
Shall I do troubly be father in in
person and the sumport and your crown's womb:
And all too much that a doint in honest:
Thou hast she both not one of the first tears are so.

JULIET:
My good lord-hearth in honours; stays upon my grow!
My name, whilst yet metrlow.

SEY:
Spittous since with him of the banished, must he will were between
Marcius of your loving; and there's a never drink.

DUCHESS OF YORK:
When I was not ever beheld the winds and potter bastards,
With raising 'kins of mine of a thearte
An warriors and none, but to interpheer in r

## functionize 

In [48]:
def iteration(embed_dim,rnn_units,epochs):
  dataset = sequences.map(split_input_target)
  dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

  vocab_size = len(vocab)
  embedding_dim = embed_dim
  rnn_units = rnn_units

  model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

  model.summary()

  model.compile(optimizer='adam', loss=loss)

  checkpoint_dir = './training_checkpoints'
  checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")
  checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_prefix,
      save_weights_only=True)

  history = model.fit(dataset, epochs=epochs, callbacks=[checkpoint_callback])

  tf.train.latest_checkpoint(checkpoint_dir)

  print("\n---\n")

  model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

  model.load_weights(tf.train.latest_checkpoint(checkpoint_dir)).expect_partial()

  model.build(tf.TensorShape([1, None]))

  model.summary()

  print(generate_text(model, start_string=u"ROMEO: "))

  print("\n---\n")

  print(generate_text(model, start_string=u"Whence"))

  print("\n---\n")

  print(generate_text(model, start_string=u"Love"))

#RNN Variations

## 256 512 10

In [49]:
iteration(256,512,10)

Model: "sequential_10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_10 (GRU)                 (64, None, 512)           1182720   
_________________________________________________________________
dense_10 (Dense)             (64, None, 65)            33345     
Total params: 1,232,705
Trainable params: 1,232,705
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 6s 25ms/step - loss: 3.1789
Epoch 2/10
172/172 [==============================] - 5s 25ms/step - loss: 2.1180
Epoch 3/10
172/172 [==============================] - 5s 25ms/step - loss: 1.8266
Epoch 4/10
172/172 [==============================] - 5s 25ms/step - loss: 1.6541
Epoch 5/10
172/172 [=========================

##256 256 10

In [50]:
iteration(256,256,10)

Model: "sequential_12"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_12 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_12 (GRU)                 (64, None, 256)           394752    
_________________________________________________________________
dense_12 (Dense)             (64, None, 65)            16705     
Total params: 428,097
Trainable params: 428,097
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 5s 15ms/step - loss: 3.1883
Epoch 2/10
172/172 [==============================] - 3s 14ms/step - loss: 2.1749
Epoch 3/10
172/172 [==============================] - 3s 14ms/step - loss: 1.9203
Epoch 4/10
172/172 [==============================] - 3s 14ms/step - loss: 1.7662
Epoch 5/10
172/172 [=============================

##256 128 10

In [51]:
iteration(256,128,10)

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_14 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_14 (GRU)                 (64, None, 128)           148224    
_________________________________________________________________
dense_14 (Dense)             (64, None, 65)            8385      
Total params: 173,249
Trainable params: 173,249
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 4s 11ms/step - loss: 3.2376
Epoch 2/10
172/172 [==============================] - 3s 11ms/step - loss: 2.2159
Epoch 3/10
172/172 [==============================] - 3s 10ms/step - loss: 1.9946
Epoch 4/10
172/172 [==============================] - 3s 10ms/step - loss: 1.8692
Epoch 5/10
172/172 [=============================

##256 64 10

In [52]:
iteration(256,64,10)

Model: "sequential_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_16 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_16 (GRU)                 (64, None, 64)            61824     
_________________________________________________________________
dense_16 (Dense)             (64, None, 65)            4225      
Total params: 82,689
Trainable params: 82,689
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 4s 10ms/step - loss: 3.3776
Epoch 2/10
172/172 [==============================] - 3s 9ms/step - loss: 2.3279
Epoch 3/10
172/172 [==============================] - 3s 9ms/step - loss: 2.1287
Epoch 4/10
172/172 [==============================] - 3s 9ms/step - loss: 2.0076
Epoch 5/10
172/172 [==============================] - 

#Embed Variations

##64 1024 10

In [57]:
iteration(64,1024,10)

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_26 (Embedding)     (64, None, 64)            4160      
_________________________________________________________________
gru_26 (GRU)                 (64, None, 1024)          3348480   
_________________________________________________________________
dense_26 (Dense)             (64, None, 65)            66625     
Total params: 3,419,265
Trainable params: 3,419,265
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 10s 46ms/step - loss: 3.4719
Epoch 2/10
172/172 [==============================] - 9s 47ms/step - loss: 2.2397
Epoch 3/10
172/172 [==============================] - 9s 47ms/step - loss: 1.9382
Epoch 4/10
172/172 [==============================] - 9s 48ms/step - loss: 1.7263
Epoch 5/10
172/172 [========================

##128 1024 10

In [58]:
iteration(128,1024,10)

Model: "sequential_28"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_28 (Embedding)     (64, None, 128)           8320      
_________________________________________________________________
gru_28 (GRU)                 (64, None, 1024)          3545088   
_________________________________________________________________
dense_28 (Dense)             (64, None, 65)            66625     
Total params: 3,620,033
Trainable params: 3,620,033
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 10s 49ms/step - loss: 3.3989
Epoch 2/10
172/172 [==============================] - 10s 50ms/step - loss: 2.1682
Epoch 3/10
172/172 [==============================] - 10s 50ms/step - loss: 1.8594
Epoch 4/10
172/172 [==============================] - 10s 51ms/step - loss: 1.6645
Epoch 5/10
172/172 [=====================

##512 1024 10

In [53]:
iteration(512,1024,10)

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_18 (Embedding)     (64, None, 512)           33280     
_________________________________________________________________
gru_18 (GRU)                 (64, None, 1024)          4724736   
_________________________________________________________________
dense_18 (Dense)             (64, None, 65)            66625     
Total params: 4,824,641
Trainable params: 4,824,641
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 14s 67ms/step - loss: 3.1904
Epoch 2/10
172/172 [==============================] - 13s 68ms/step - loss: 1.9692
Epoch 3/10
172/172 [==============================] - 13s 70ms/step - loss: 1.6783
Epoch 4/10
172/172 [==============================] - 13s 70ms/step - loss: 1.5203
Epoch 5/10
172/172 [=====================

##1024 1024 10

In [59]:
iteration(1024,1024,10)

Model: "sequential_30"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_30 (Embedding)     (64, None, 1024)          66560     
_________________________________________________________________
gru_30 (GRU)                 (64, None, 1024)          6297600   
_________________________________________________________________
dense_30 (Dense)             (64, None, 65)            66625     
Total params: 6,430,785
Trainable params: 6,430,785
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
172/172 [==============================] - 17s 86ms/step - loss: 2.8432
Epoch 2/10
172/172 [==============================] - 16s 87ms/step - loss: 1.8120
Epoch 3/10
172/172 [==============================] - 16s 88ms/step - loss: 1.5617
Epoch 4/10
172/172 [==============================] - 16s 89ms/step - loss: 1.4467
Epoch 5/10
172/172 [=====================

#Epoch Variations

##256 1024 5

In [60]:
iteration(256,1024,5)

Model: "sequential_32"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_32 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_32 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_32 (Dense)             (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/5
172/172 [==============================] - 11s 54ms/step - loss: 3.3033
Epoch 2/5
172/172 [==============================] - 10s 54ms/step - loss: 2.0494
Epoch 3/5
172/172 [==============================] - 10s 55ms/step - loss: 1.7412
Epoch 4/5
172/172 [==============================] - 11s 55ms/step - loss: 1.5672
Epoch 5/5
172/172 [==========================

##256 1024 20

In [54]:
iteration(256,1024,20)

Model: "sequential_20"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_20 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_20 (Dense)             (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
172/172 [==============================] - 12s 57ms/step - loss: 3.2602
Epoch 2/20
172/172 [==============================] - 11s 59ms/step - loss: 2.0678
Epoch 3/20
172/172 [==============================] - 11s 60ms/step - loss: 1.7591
Epoch 4/20
172/172 [==============================] - 12s 61ms/step - loss: 1.5811
Epoch 5/20
172/172 [=====================

##256 1024 30

In [55]:
iteration(256,1024,30)

Model: "sequential_22"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_22 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_22 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_22 (Dense)             (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/30
172/172 [==============================] - 12s 56ms/step - loss: 3.2333
Epoch 2/30
172/172 [==============================] - 11s 56ms/step - loss: 2.0608
Epoch 3/30
172/172 [==============================] - 11s 57ms/step - loss: 1.7494
Epoch 4/30
172/172 [==============================] - 11s 58ms/step - loss: 1.5718
Epoch 5/30
172/172 [=====================

##256 1024 40

In [56]:
iteration(256,1024,40)

Model: "sequential_24"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_24 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_24 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_24 (Dense)             (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
172/172 [==============================] - 11s 54ms/step - loss: 3.2118
Epoch 2/40
172/172 [==============================] - 10s 55ms/step - loss: 2.0447
Epoch 3/40
172/172 [==============================] - 11s 55ms/step - loss: 1.7278
Epoch 4/40
172/172 [==============================] - 11s 56ms/step - loss: 1.5607
Epoch 5/40
172/172 [=====================

##256 1024 50

In [61]:
iteration(256,1024,50)

Model: "sequential_34"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_34 (Embedding)     (64, None, 256)           16640     
_________________________________________________________________
gru_34 (GRU)                 (64, None, 1024)          3938304   
_________________________________________________________________
dense_34 (Dense)             (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50
172/172 [==============================] - 11s 53ms/step - loss: 3.2904
Epoch 2/50
172/172 [==============================] - 10s 54ms/step - loss: 2.0477
Epoch 3/50
172/172 [==============================] - 10s 54ms/step - loss: 1.7375
Epoch 4/50
172/172 [==============================] - 10s 55ms/step - loss: 1.5625
Epoch 5/50
172/172 [=====================

## MAX

In [63]:
iteration(1024,1024,40)

Model: "sequential_38"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_38 (Embedding)     (64, None, 1024)          66560     
_________________________________________________________________
gru_38 (GRU)                 (64, None, 1024)          6297600   
_________________________________________________________________
dense_38 (Dense)             (64, None, 65)            66625     
Total params: 6,430,785
Trainable params: 6,430,785
Non-trainable params: 0
_________________________________________________________________
Epoch 1/40
172/172 [==============================] - 16s 83ms/step - loss: 3.0240
Epoch 2/40
172/172 [==============================] - 16s 85ms/step - loss: 1.8548
Epoch 3/40
172/172 [==============================] - 16s 86ms/step - loss: 1.5906
Epoch 4/40
172/172 [==============================] - 16s 86ms/step - loss: 1.4714
Epoch 5/40
172/172 [=====================